In [1]:
%load_ext autoreload
%autoreload 2
import pickle
from MARBLE import utils, plotting, net, preprocessing, postprocessing, geometry

import matplotlib.pyplot as plt

# Visualise a single condition in a single session

In [2]:
#pos, vel, time, cond_labels = pickle.load(open('../outputs/spiking_data/raw_data_session_1.pkl','rb'))

data = []
for i in range(2):
    data.append(pickle.load(open('../outputs/spiking_data/data_object_session_{}.pkl'.format(i),'rb')))

In [ ]:
from torch_geometric.data import Batch
from torch_geometric.transforms import RandomNodeSplit
batch = Batch.from_data_list(data, exclude_keys=['L','gauges','dist','gamma', 'manifold', 'emb','clusters'])
split = RandomNodeSplit(split='train_rest', num_val=0.1, num_test=0.1)
split(batch)
    
batch = preprocessing(batch, vector=False, compute_laplacian=True, n_evec=128)

---- Embedding dimension: 5
---- Signal dimension: 5

Vector computations are disabled

---- Computing kernels ... Done 
---- Computing eigendecomposition ... 

# Train model on all conditions and sessions

In [ ]:
par = {'epochs': 150, #optimisation epochs
       'order': 2, #order of derivatives
       'hidden_channels': 32, #number of internal dimensions in MLP
       'out_channels': 8,
       'inner_product_features': False,
       'diffusion': True,
      }
    
model = net(batch, par=par)
    
model.run_training(batch, use_best=True, outdir='../outputs/spiking_data/all_sessions') 

In [ ]:
data = model.evaluate(data)
data = postprocessing(data)

In [ ]:
conditions = ['DownLeft','Left','UpLeft','Up','UpRight','Right','DownRight']
plotting.embedding(data, data.y.numpy().astype(int), titles=conditions)

In [ ]:
plt.imshow(data.dist); plt.colorbar()  

In [ ]:
emb_MDS, _ = geometry.embed(data.dist, embed_typ = 'MDS')
labels = [i for i in range(len(conditions))]
plotting.embedding(emb_MDS, labels, titles=conditions, s=30, alpha=1)

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
rates = pickle.load(open('../outputs/spiking_data/rate_data_1ms.pkl','rb'))       

# definingf the set of conditions     
conditions=['DownLeft','Left','UpLeft','Up','UpRight','Right','DownRight']    
    
# list of days
days = rates.keys()
    
# define some parameters
pca_n = 5
      
# storing all distance matrices
embeddings = []
distance_matrices = []
    

times = [];
all_condition_labels = [];
    
# loop over each day
for day in days:
        
        # first stack all trials from that day together and fit pca
        print(day)
        pos = []            
        # loop over each condition on that day
        for c, cond in enumerate(conditions):

            # go cue at 500ms
            data = rates[day][cond][:,500:,:]
            
            # loop over all trials
            for trial in data:
                # store each trial as time x channels
                pos.append(trial)
              
        # stacking all trials into a single array (time x channels)
        pos = np.vstack(pos)
        print(pos.shape)
        
        # fit PCA to all data across all conditions on a given day simultaneously
        pca = PCA(n_components=pca_n)
        pca.fit(pos)     
        print(pca.explained_variance_ratio_)  

In [ ]:
rates[day][cond].shape